In [4]:
## LIBRARIES

# Import libraries
import pandas as pd
import numpy as np
import openai 
from openai import OpenAI
import asyncio
from IPython.display import display, HTML


# Define absolute python path
import sys
sys.path.insert(0, '/Users/giorgiobolchi2/Documents/JRC/LLM') 



## DATA

# Load NACE data
from Data.NACEdata import NACElevel0, NACElevel1, NACElevel2, NACElevel3     

# Import all target data (target_code + target_content)
targets_pd = pd.read_csv('/Users/giorgiobolchi2/Documents/JRC/LLM/Data/XLSX_target_data_v1.2_LLM.csv', sep=";")



In [5]:

# Set API token
TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6ImY3NDcxY2MwLTE5NWItNDZhOS1hNjc1LTM0Y2EzY2E1MmI2YSIsImlzcyI6ImdwdGpyYyIsImlhdCI6MTczNzQ1NjU4NywiZXhwIjoxNzUwNDY0MDAwLCJpc19yZXZva2VkIjpmYWxzZSwiYWNjb3VudF9pZCI6IlFXeGxjeTVTUlVKRlEwQmxlSFF1WldNdVpYVnliM0JoTG1WMVMyNXZkMU5FUjNNPSIsInVzZXJuYW1lIjoiQWxlcy5SRUJFQ0BleHQuZWMuZXVyb3BhLmV1IiwicHJvamVjdF9pZCI6Iktub3dTREdzIiwiZGVwYXJ0bWVudCI6IkpSQy5ELjEiLCJxdW90YXMiOlt7Im1vZGVsX25hbWUiOiJncHQtNG8iLCJleHBpcmF0aW9uX2ZyZXF1ZW5jeSI6ImRhaWx5IiwidmFsdWUiOjQwMDAwMH0seyJtb2RlbF9uYW1lIjoiZ3B0LTM1LXR1cmJvLTExMDYiLCJleHBpcmF0aW9uX2ZyZXF1ZW5jeSI6ImRhaWx5IiwidmFsdWUiOjQwMDAwMH0seyJtb2RlbF9uYW1lIjoiZ3B0LTQtMTEwNiIsImV4cGlyYXRpb25fZnJlcXVlbmN5IjoiZGFpbHkiLCJ2YWx1ZSI6NDAwMDAwfV0sImFjY2Vzc19ncm91cHMiOlt7ImlkIjoiNDJkMTUzMTAtNzdlZS00MDgyLTk1NzQtZDZhYTk5YzliZDgzIiwiYWNjZXNzX2dyb3VwIjoiZ2VuZXJhbCJ9XX0.0S3jlmjrBLGPwQaJ4U5F20DpacvyDcK_NsS412BX_Pw"
openai.api_base = "https://api-gpt.jrc.ec.europa.eu/v1"
openai.api_key = TOKEN

client = OpenAI(
        api_key = TOKEN,
        base_url = "https://api-gpt.jrc.ec.europa.eu/v1/"    )


In [6]:

from typing import List
from scipy import spatial

def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding


def distances_from_embeddings(
    query_embedding: List[float],
    embeddings: List[List[float]],
    distance_metric="cosine",
) -> List[List]:
    """Return the distances between a query embedding and a list of embeddings."""
    distance_metrics = {
        "cosine": spatial.distance.cosine,
        "L1": spatial.distance.cityblock,
        "L2": spatial.distance.euclidean,
        "Linf": spatial.distance.chebyshev,
    }
    distances = [
        distance_metrics[distance_metric](query_embedding, embedding)
        for embedding in embeddings
    ]
    return distances


In [7]:
async def get_chat_response(
    user_request: str
):
    try:
        messages = [
            {"role": "user", "content": user_request},
        ]

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            seed=None,
        )

        response_content = response.choices[0].message.content
        system_fingerprint = response.system_fingerprint
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = response.usage.total_tokens - response.usage.prompt_tokens

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

def calculate_average_distance(responses):
    """
    This function calculates the average distance between the embeddings of the responses.
    The distance between embeddings is a measure of how similar the responses are.
    """
    # Calculate embeddings for each response
    response_embeddings = [get_embedding(response) for response in responses]

    # Compute distances between the first response and the rest
    distances = distances_from_embeddings(response_embeddings[0], response_embeddings[1:])

    # Calculate the average distance
    average_distance = sum(distances) / len(distances)

    # Return the average distance
    return average_distance


In [8]:
TA = ['TA1']
target_subset = targets_pd[targets_pd['target_code'].str.contains(TA[0])] # import TA1 as panda dataframe 
topic = target_list = [f"{row['target_code']}: {row['target_content']}" for index, row in target_subset.iterrows()] #make as list
user_request = f"Summarize the content of: {topic}."

responses = []


async def get_response(i):
    print(f'Output {i + 1}\n{"-" * 10}')
    response = await get_chat_response(
        user_request=user_request
    )
    return response


responses = await asyncio.gather(*[get_response(i) for i in range(5)])
average_distance = calculate_average_distance(responses)
print(f"The average similarity between responses is: {average_distance}")

Output 1
----------
An error occurred: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}
Output 2
----------
An error occurred: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}
Output 3
----------
An error occurred: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}
Output 4
----------
An error occurred: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}
Output 5
----------
An error occurred: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_req

AttributeError: 'NoneType' object has no attribute 'replace'